In [1]:
import tensorflow as tf
gpu_devices = tf.config.experimental.list_physical_devices('GPU')
for device in gpu_devices:
    tf.config.experimental.set_memory_growth(device, True)

2021-12-23 10:25:16.806033: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-23 10:25:16.817481: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-23 10:25:16.818221: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


In [2]:
import sys, os
cwd = os.getcwd()
sys.path.append(cwd + '/../.') 

import math
import argparse
import tensorflow as tf
import numpy as np
from sklearn.metrics import f1_score
from src.EvaluationLoop import EvaluationLoop
from src.training_loop2 import QueryStrategy
from data.datasets import Datasets, Encodings
from sklearn.metrics import f1_score
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import train_test_split
from transformers import TFBertForSequenceClassification, TFTrainer, TFTrainingArguments, BertTokenizerFast

In [3]:
name = 'final'
dataset = 'reuters'

In [4]:
    data_X = np.load(f'../data/datasets/{dataset}_X.npy')
    data_y = np.load(f'../data/datasets/{dataset}_y.npy')

In [5]:
data_X.shape

(8759,)

In [6]:
data_y.shape

(8759,)

In [7]:
    sss = StratifiedShuffleSplit(n_splits=5, test_size=0.5, random_state=0)
    sss.get_n_splits(data_X, data_y)

5

In [9]:
    i = 0
    for train_index, test_index in sss.split(data_X, data_y):
        print(f'Model #{i} ...')
        X_train, X_test = data_X[train_index], data_X[test_index]
        y_train, y_test = data_y[train_index], data_y[test_index]
        
        X_used = X_train
        y_used = y_train
        
        break;

Model #0 ...


In [10]:
print(len(X_used), len(X_test))

4379 4380


In [11]:
X_bert_train, X_bert_eval, y_bert_train, y_bert_eval = train_test_split(X_used, y_used, test_size=0.10, random_state=42, stratify=y_used)

In [13]:
print('tokenize ...')
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

tokenize ...


In [14]:
        train_encodings = tokenizer(X_bert_train.tolist(), max_length=350, truncation=True, padding=True)
        val_encodings = tokenizer(X_bert_eval.tolist(), max_length=350, truncation=True, padding=True)
        test_encodings = tokenizer(X_test.tolist(), max_length=350, truncation=True, padding=True)

In [15]:
        del X_bert_train
        del X_bert_eval
        del X_test
        del tokenizer

In [16]:
        train_dataset = tf.data.Dataset.from_tensor_slices((
            dict(train_encodings),
            y_bert_train
        ))
        val_dataset = tf.data.Dataset.from_tensor_slices((
            dict(val_encodings),
            y_bert_eval
        ))
        test_dataset = tf.data.Dataset.from_tensor_slices((
            dict(test_encodings),
            y_test
        ))

2021-12-23 10:27:48.864368: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-12-23 10:27:48.865715: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-23 10:27:48.870745: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-23 10:27:48.871246: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA 

In [17]:
        print(len(set(y_test)))
        print(len(set(y_bert_eval)))
        print(len(set(y_bert_train)))

9
9
9


In [21]:
        del train_encodings
        del val_encodings
        del test_encodings

In [26]:
        training_args = TFTrainingArguments(
                    evaluation_strategy= "steps",
                    output_dir='./bert_models2',  
                    num_train_epochs=5,              # total number of training epochs
                    per_device_train_batch_size=16,  # batch size per device during training
                    per_device_eval_batch_size=16,   # batch size for evaluation
                    #warmup_steps=500,                # number of warmup steps for learning rate scheduler
                    weight_decay=0.01,               # strength of weight decay
                    warmup_steps=math.ceil(len(y_bert_train) / 12),
                    logging_dir='./logs',         # directory for storing logs
                    logging_steps=2000,
                    learning_rate= 5e-5,
                    save_total_limit = 1,
                    #load_best_model_at_end= True,
                )

In [27]:
        print('Load BERT model ...')
        
        n_classes = len(set(y_bert_train))
        print(n_classes)

Load BERT model ...
9


In [28]:
        with training_args.strategy.scope():
            model = TFBertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=n_classes)

        trainer = TFTrainer(
            model=model,                         # the instantiated 🤗 Transformers model to be trained
            args=training_args,                  # training arguments, defined above
            train_dataset=train_dataset,         # training dataset
            eval_dataset=val_dataset             # evaluation dataset
        )

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [29]:
        print('Train BERT model ...')
        trainer.train()

Train BERT model ...


2021-12-23 10:30:31.028843: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:695] AUTO sharding policy will apply DATA sharding policy as it failed to apply FILE sharding policy because of the following reason: Found an unshardable source dataset: name: "TensorSliceDataset/_4"
op: "TensorSliceDataset"
input: "Placeholder/_0"
input: "Placeholder/_1"
input: "Placeholder/_2"
input: "Placeholder/_3"
attr {
  key: "Toutput_types"
  value {
    list {
      type: DT_INT32
      type: DT_INT32
      type: DT_INT32
      type: DT_INT64
    }
  }
}
attr {
  key: "output_shapes"
  value {
    list {
      shape {
        dim {
          size: 350
        }
      }
      shape {
        dim {
          size: 350
        }
      }
      shape {
        dim {
          size: 350
        }
      }
      shape {
      }
    }
  }
}

2021-12-23 10:30:31.306589: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2021-1

KeyboardInterrupt: 

In [1]:
import numpy as np
from sklearn.model_selection import train_test_split
from transformers import DistilBertTokenizerFast, BertTokenizerFast

In [2]:
data_set= 'app_store'
encoding = 'sbert'
use_random = False

rng = '_rnd' if use_random else ''

In [3]:
data_X = np.load('./../data/datasets/%s_X.npy' % data_set)
data_y = np.load('./../data/datasets/%s_y.npy' % data_set)

used_index_meta_list = np.load('./../eval_proxy/res/%s_%s%s/used_training_index.npy' % (data_set, encoding, rng), allow_pickle=True)

In [4]:
# get data
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit

sss = StratifiedShuffleSplit(n_splits=5, test_size=0.5, random_state=0)
sss.get_n_splits(data_X, data_y)

i = 0
for train_index, test_index in sss.split(data_X, data_y):
    X_train, X_test = data_X[train_index], data_X[test_index]
    y_train, y_test = data_y[train_index], data_y[test_index]
    break;

X_test = X_test.tolist()

In [5]:
from sklearn.utils import shuffle
X_train, y_train = shuffle(X_train, y_train, random_state=0)

In [6]:
# get pool
i = 0
used_index = used_index_meta_list[i][1]

X_pool = []
y_pool = []
for i in used_index:
    X_pool.append(X_train[i])
    y_pool.append(y_train[i])

In [7]:
X_pool = X_train[:630].tolist()
y_pool = y_train[:630].tolist()

In [8]:
from sklearn.utils import shuffle

In [9]:
len(X_pool)

630

In [10]:
X_train = None
y_train = None

In [11]:
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

In [12]:
X_bert_train, X_bert_eval, y_bert_train, y_bert_eval = train_test_split(X_pool, y_pool, test_size=0.10, random_state=42, stratify=y_pool)

In [13]:
len(X_bert_train), len(X_bert_eval)

(567, 63)

In [14]:
train_encodings = tokenizer(X_bert_train, truncation=True, padding=True)
val_encodings = tokenizer(X_bert_eval, truncation=True, padding=True)
test_encodings = tokenizer(X_test, truncation=True, padding=True)

In [15]:
import tensorflow as tf

train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    y_bert_train
))
val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(val_encodings),
    y_bert_eval
))
test_dataset = tf.data.Dataset.from_tensor_slices((
    dict(test_encodings),
    y_test
))

2021-11-02 09:10:38.560679: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-02 09:10:38.571587: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-02 09:10:38.572367: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-02 09:10:38.573534: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

In [16]:
train_encodings=None
val_encodings=None
test_encodings=None

In [19]:
import math

from transformers import TFBertForSequenceClassification, TFTrainer, TFTrainingArguments


training_args = TFTrainingArguments(
            evaluation_strategy= "steps",
            output_dir='./bert_models',  
            num_train_epochs=10,              # total number of training epochs
            per_device_train_batch_size=16,  # batch size per device during training
            per_device_eval_batch_size=32,   # batch size for evaluation
            #warmup_steps=500,                # number of warmup steps for learning rate scheduler
            weight_decay=0.01,               # strength of weight decay
            warmup_steps=math.ceil(len(y_bert_train) / 16),
            logging_dir='./logs',         # directory for storing logs
            logging_steps=2000,
            learning_rate= 5e-5,
            save_total_limit = 1,
            #load_best_model_at_end= True,
        )

with training_args.strategy.scope():
    model = TFBertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=3)

trainer = TFTrainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset             # evaluation dataset
)

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/opt/conda/lib/python3.9/site-packages/transformers/trainer_tf.py:109: FutureWarning: The class `TFTrainer` is deprecated and will be removed in version 5 of Transformers. We recommend using native Keras instead, by calling methods like `fit()` and `predict()` directly on the model object. Detailed examples of the Keras style can be found in our examples at https://github.com/huggingface/transformers/tree/master/examples/tensorflow
  warnings.warn(


In [20]:
trainer.train()

2021-11-02 09:07:29.144288: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:695] AUTO sharding policy will apply DATA sharding policy as it failed to apply FILE sharding policy because of the following reason: Found an unshardable source dataset: name: "TensorSliceDataset/_3"
op: "TensorSliceDataset"
input: "Placeholder/_0"
input: "Placeholder/_1"
input: "Placeholder/_2"
attr {
  key: "Toutput_types"
  value {
    list {
      type: DT_INT32
      type: DT_INT32
      type: DT_INT32
    }
  }
}
attr {
  key: "output_shapes"
  value {
    list {
      shape {
        dim {
          size: 417
        }
      }
      shape {
        dim {
          size: 417
        }
      }
      shape {
      }
    }
  }
}

2021-11-02 09:07:29.193613: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


KeyboardInterrupt: 

In [18]:
model.summary()

Model: "tf_bert_for_sequence_classification"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  109482240 
_________________________________________________________________
dropout_37 (Dropout)         multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  2307      
Total params: 109,484,547
Trainable params: 109,484,547
Non-trainable params: 0
_________________________________________________________________


In [19]:
preds = trainer.model.predict(test_dataset.batch(8))
p_softmax = tf.nn.softmax(preds.logits, axis=1).numpy()

2021-10-19 14:33:29.974581: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:695] AUTO sharding policy will apply DATA sharding policy as it failed to apply FILE sharding policy because of the following reason: Found an unshardable source dataset: name: "TensorSliceDataset/_3"
op: "TensorSliceDataset"
input: "Placeholder/_0"
input: "Placeholder/_1"
input: "Placeholder/_2"
attr {
  key: "Toutput_types"
  value {
    list {
      type: DT_INT32
      type: DT_INT32
      type: DT_INT64
    }
  }
}
attr {
  key: "output_shapes"
  value {
    list {
      shape {
        dim {
          size: 1790
        }
      }
      shape {
        dim {
          size: 1790
        }
      }
      shape {
      }
    }
  }
}



In [20]:
y_pred = p_softmax.argmax(axis=1)

In [21]:
from sklearn.metrics import f1_score
f1_score(y_test, y_pred, average='micro'), f1_score(y_test, y_pred, average='macro')

(0.8620150187734669, 0.8224995047948452)

In [24]:
# BERT
from sklearn.metrics import f1_score
f1_score(y_test, y_pred, average='micro'), f1_score(y_test, y_pred, average='macro')

(0.8563829787234043, 0.8183368300130778)

In [23]:
from sklearn.metrics import f1_score
f1_score(y_test, y_pred, average='micro'), f1_score(y_test, y_pred, average='macro')

(0.8504380475594493, 0.8105004485531732)

In [41]:
from sklearn.metrics import f1_score
f1_score(y_test, y_pred, average='micro'), f1_score(y_test, y_pred, average='macro')

(0.853566958698373, 0.8173382270993904)